In [0]:
filenum =100
text = []
files = os.listdir('./randomsong')
for file in files[:filenum]:
  with open('./randomsong/' + file) as f:
    temp = f.read().split('\n')
    text.append(temp)

input_text = []
target_text = []

maxlen=0
for t in text:
  for i in range(len(t)-1):
    input_text.append(t[i] + ' \n')
    target_text.append(' \t' + t[i+1] + ' \n')
    if maxlen < len(t[i+1].split(' ')):
      maxlen = len(t[i+1].split(' '))

text = ''
files = os.listdir('./randomsong')
for file in files[:filenum]:
  with open('./randomsong/' + file) as f:
    text += f.read()
    
text = text.replace('\n', ' ')
text = text.split()
text += ' \n \t'
words = sorted(list(set(text)))

ind_w = dict([(i,c) for i,c in enumerate(words)])
word_i = dict([(c,i) for i,c in enumerate(words)])

x = np.zeros((len(input_text), maxlen, len(words)), dtype='float32')
y = np.zeros((len(input_text), maxlen, len(words)), dtype='float32')
y_next = np.zeros((len(input_text), maxlen, len(words)), dtype='float32')

for i,(input, target) in enumerate(zip(input_text, target_text)):
  input = input.split()
  target = target.split()
  for j, (inp,tar) in enumerate(zip(input,target)):
    x[i, j, word_i[inp]] = 1
    y[i, j, word_i[tar]] = 1
    if j > 0:
      y_next[i,j-1, word_i[tar]] = 1
      

encoder_inputs = Input(shape=(None, len(words)))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, len(words)))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
      
decoder_dense = Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([x,y], y_next, batch_size=64, epochs=100,validation_split=0.2)





In [0]:
latent_dim=256
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


def decode_seq(input_seq):
  states_value = encoder_model.predict(input_seq)
  target_seq = np.zeros((1,1, len(words)))
  target_seq[0,0, word_i['\t']] = 1.

  stop_condition = False
  decoded_sentense = ' '
  while not stop_condition:
    output_tokens , h, c = decoder_model.predict(\
                         [target_seq] + states_value)
    sampled_token_index = np.argmax(output_tokens[0,-1,:])
    sampled_word = ind_w[sampled_token_index]
    decoded_sentense += str(' ') + sampled_word
    if (sampled_word == '\n' or (len(decoded_sentense) > maxlen and target_seq[0,0, word_i['\t']] != 1)):
      stop_condition = True
      
    target_seq = np.zeros((1,1, len(words)))
    target_seq[0,0, sampled_token_index] = 1
    
    state_value = [h,c]
    
  return decoded_sentense

In [0]:
for i in range(100):
  input_seq = x[i:i+1]
  decoded_sentense = decode_seq(input_seq)
  
  print('\n' +str(i))
  print(input_text[i].replace('\n', ''))
  print(input_text[i+1].replace('\n', ''))
  print(decoded_sentense.replace('\t', ''))